In [2]:
import os
os.chdir('../')
%pwd

'/Users/sanghyuk/myProjects/mlproject02'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [4]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories
from pathlib import Path

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        
        create_directories([Path(config.root_dir)])
        
        return ModelEvaluationConfig(
            root_dir=Path(config.root_dir),
            data_path=Path(config.data_path),
            model_path=Path(config.model_path),
            tokenizer_path=Path(config.tokenizer_path),
            metric_file_name=Path(config.metric_file_name)
        )

In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk, load_metric
import torch
import pandas as pd
from tqdm import tqdm

In [15]:
class ModelEvalution:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        """
        split the dataset into smaller batches that we can process simultaneously.
        Yield successive batch-sized chunks from list_of_elements.
        """
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i+batch_size]


    def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer,
                                    device, batch_size=16,
                                    column_text="article",
                                    column_summary="highlights"):
        article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)
        ):

            inputs = tokenizer(article_batch, max_length=1024, truncation=True,
                            padding='max_length', return_tensors="pt")

            # parameter for length penalty ensures that the model does not generate seqeunces too long
            summaries = model.generate(input_ids=inputs['input_ids'].to(device),
                                    attention_mask=inputs['attention_mask'].to(device),
                                    length_penalty=0.8, num_beams=8, max_length=128)

            # Finally, we decode the generated texts,
            # replace the token, and add the decoded texts with the references to the metric
            decoded_summaries = [
                tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
                for s in summaries
            ]

            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

            metric.add_batch(predictions=decoded_summaries, references=target_batch)

        # Finally compute and return the ROUGE scores
        score = metric.compute()
        return score
    
    def evaluate(self):        
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(self.device)
        
        # load the dataset
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        rouge_names = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
        
        rouge_metric = load_metric('rouge')
        
                
        # calculate metric on test data
        score = self.calculate_metric_on_test_ds(
            dataset=dataset_samsum_pt['test'][:10],
            metric=rouge_metric, 
            model=model_pegasus, 
            tokenizer=tokenizer,
            device=self.device,
            batch_size=2,
            column_text='dialogue',
            column_summary='summary'
        )
        
        rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
        
        df = pd.DataFrame(rouge_dict, index=['pegasus'])
        df.to_csv(self.config.metric_file_name, index=False)

In [9]:
try:
    config = ConfigurationManager()
    model_evaluation = ModelEvaluation(config.get_model_evaluation_config())
    model_evaluation.evaluate()
except Exception as e:
    raise e

{'a': 1, 'b': 2, 'c': 3, 'd': 4}